In [1]:
import os
import time
from math import floor, ceil

import numpy
from keras import layers, metrics
from keras import models, optimizers
from keras.applications import VGG16
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from utils import *
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

train_dir = '/home/snowflake/Documents/nowe_trendy/train'
test_dir = '/home/snowflake/Documents/nowe_trendy/test'

path, dirs, files = os.walk(train_dir).__next__()

val_size = 0.15
nTrain = sum([len(files) for r, d, files in os.walk(train_dir)]) - len(dirs)
nVal = ceil(nTrain * val_size)
path, dirs, files = os.walk(test_dir).__next__()
nTest = sum([len(files) for r, d, files in os.walk(test_dir)]) - len(dirs)
nClasses = len(dirs)
batch_size = 20
dense_size = 256
epochs = 10

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_features = np.load(open("/media/snowflake/Data/train_features.dat", "rb"))
train_labels = np.load(open("/media/snowflake/Data/train_labels.dat", "rb"))
validation_features = np.load(open("/media/snowflake/Data/validation_features.dat", "rb"))
validation_labels = np.load(open("/media/snowflake/Data/validation_labels.dat", "rb"))
test_features = np.load(open("/media/snowflake/Data/test_features.dat", "rb"))
test_labels = np.load(open("/media/snowflake/Data/test_labels.dat", "rb"))

In [50]:
epochs = 40
batch_size = 64
dense_size = 1024
lr=0.001
momentum=0.6
#2e-4

In [51]:
print("**********Configure model for training************")
model = models.Sequential()
model.add(layers.Dense(dense_size, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(nClasses, activation='softmax'))

model.compile(optimizer=optimizers.SGD(lr, momentum),
              loss='categorical_crossentropy',
              metrics=[metrics.categorical_accuracy])

**********Configure model for training************


In [52]:
print("**********Start of training************")
history = model.fit(train_features,
                    train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_features, validation_labels))

**********Start of training************
Train on 15975 samples, validate on 3413 samples
Epoch 1/40
15975/15975 [==============================] - 46s 3ms/step - loss: 2.2920 - categorical_accuracy: 0.2198 - val_loss: 2.0759 - val_categorical_accuracy: 0.3117
Epoch 2/40
15975/15975 [==============================] - 45s 3ms/step - loss: 2.0112 - categorical_accuracy: 0.3234 - val_loss: 1.8860 - val_categorical_accuracy: 0.3683
Epoch 3/40
15975/15975 [==============================] - 46s 3ms/step - loss: 1.8775 - categorical_accuracy: 0.3679 - val_loss: 1.8039 - val_categorical_accuracy: 0.4005
Epoch 4/40
15975/15975 [==============================] - 46s 3ms/step - loss: 1.7749 - categorical_accuracy: 0.4038 - val_loss: 1.7742 - val_categorical_accuracy: 0.4011
Epoch 5/40
15975/15975 [==============================] - 46s 3ms/step - loss: 1.7058 - categorical_accuracy: 0.4307 - val_loss: 1.6976 - val_categorical_accuracy: 0.4240
Epoch 6/40
15975/15975 [==============================] 

In [53]:
print("**********End of training************\n\n")

ground_truth = [np.where(r == 1)[0][0] for r in test_labels]

predictions = model.predict_classes(test_features)
prob = model.predict(test_features)

# errors = nVal - accuracy_score(ground_truth, predictions, normalize=False)
errors = np.where(predictions != ground_truth)[0]
accuracy = accuracy_score(ground_truth, predictions)
precision, recall, f_score, support = score(ground_truth, predictions, average='weighted')

print("No of errors = {}/{}".format(len(errors), nTest))
print("Accuracy: {}".format(accuracy))
print("Weighted average Precision: {}".format(precision))
print("Weighted average Recall: {}".format(recall))
print("Weighted average F-score: {}".format(f_score))

**********End of training************


No of errors = 711/1809
Accuracy: 0.6095551894563427
Weighted average Precision: 0.6141485755263442
Weighted average Recall: 0.6095551894563427
Weighted average F-score: 0.6055553305460621
